# A2I Learning Lab


## 1. Overview

**Terminology**

- HIL: Human in Loop

## 2. Create Labeling Workforce Private Team

In this step, we create a labeling workforce private team and invite your employee to help us do the Human-in-Loop task.

1. go to AWS Sagemaker Console -> Ground Truth sub menu -> Labeling workforces -> Private -> Create Private Team:

<img src="./images/01-Create-Labeling-Workforce-Private-Team.png" width="1920"/>

2. configure the private team, you can follow detailed instruction below

<img src="./images/02-Configure-Labeling-Workforce-Private-Team.png" width="1600"/>

- Private Team Creation = Create a private team with [AWS Cognito](https://aws.amazon.com/cognito/)
- Team details:
    - Team name = ``my-labeling-team``
- Add Workers:
    - Invite new workers by email, Email address: ``alice@example.com``
    - Organization name: ``my-org``
    - Contact email: ``admin@example.com``
- Enable Notifications: we don't need this for learning.
- Click "Create Private Team" button.

3. Now you can enter your Private Team console. There is a sign-un URL your workers can log in to the HIL workspace. If you want to add more workers to your team, You can invite more people by clicking the "Invite new workers" button.

<img src="./images/03-Private-Team-GUI.png" width="1920"/>

4. As a worker, once you logged in to the HIL workspace, you will see the following GUI. Now we don't have any HIL task available yet.

<img src="./images/04-HIL-Workspace-Console.png" width="1920"/>

## 3. Create Human in Loop Workflow

in this section, we create all required AWS resources for Human in Loop Workflow, including:

1. An S3 bucket to store the HIL data
2. An IAM Role for Human Review Workflow execution
3. A Human Review Workflow definition that defines the metadata about this workflow
4. A Task template that defines the HIL task HTML UI

In [1]:
% pip install -r requirements.txt

UsageError: Line magic function `%` not found.


In [29]:
# import standard library
import typing as T
import os
import json
import time
import uuid
import dataclasses

# import 3rd party library
from pathlib_mate import Path
from s3pathlib import S3Path, context
from boto_session_manager import BotoSesManager, AwsServiceEnum

dir_here = Path(os.getcwd()).absolute()
path_task_template = dir_here / "task.liquid"
path_task_ui_html = dir_here / "task.html"
path_task_input = dir_here / "task.json"


@dataclasses.dataclass
class Lab:
    # constant attributes
    project_name: str = dataclasses.field()
    labeling_team_arn: str = dataclasses.field()
    bsm: BotoSesManager = dataclasses.field(default=None)
    path_task_template: Path = dataclasses.field(default=path_task_template)
    path_task_ui_html: Path = dataclasses.field(default=path_task_ui_html)
    path_task_input: Path = dataclasses.field(default=path_task_input)

    # derived attributes
    s3_client: T.Any = dataclasses.field(default=None)
    iam_client: T.Any = dataclasses.field(default=None)
    sm_client: T.Any = dataclasses.field(default=None)
    a2i_client: T.Any = dataclasses.field(default=None)

    def __post_init__(self):
        if self.bsm is None:
            self.bsm = BotoSesManager()
        context.attach_boto_session(self.bsm.boto_ses)

        self.s3_client = self.bsm.get_client(AwsServiceEnum.S3)
        self.iam_client = self.bsm.get_client(AwsServiceEnum.IAM)
        self.sm_client = self.bsm.get_client(AwsServiceEnum.SageMaker)
        self.a2i_client = self.bsm.get_client(AwsServiceEnum.AugmentedAIRuntime)

    @property
    def project_name_slug(self) -> str:
        return self.project_name.replace("_", "-")

    @property
    def common_tags(self) -> T.List[T.Dict[str, str]]:
        return [
            dict(
                Key="ProjectName",
                Value=self.project_name_slug,
            )
        ]

    # --- Create S3 bucket to store HIL data
    @property
    def bucket_name(self) -> str:
        return f"{self.bsm.aws_account_id}-{self.bsm.aws_region}-{self.project_name_slug}"

    @property
    def bucket_console_url(self) -> str:
        return f"https://s3.console.aws.amazon.com/s3/buckets/{self.bucket_name}?region={self.bsm.aws_region}&tab=objects"

    def step_1a_create_s3_bucket(self) -> dict:
        print("Create s3 bucket to store HIL data")
        print(f"  Preview at {self.bucket_console_url}")
        # ref: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
        response1 = self.s3_client.create_bucket(
            Bucket=self.bucket_name,
            CreateBucketConfiguration=dict(
                LocationConstraint=self.bsm.aws_region,
            ),
        )

        # grant CORS permission so HIL UI can access artifacts in S3 bucket
        # ref: https://docs.aws.amazon.com/sagemaker/latest/dg/sms-cors-update.html
        # ref: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.put_bucket_cors
        response2 = self.s3_client.put_bucket_cors(
            Bucket=self.bucket_name,
            CORSConfiguration={
                "CORSRules": [
                    {
                        "AllowedHeaders": [],
                        "AllowedMethods": ["GET"],
                        "AllowedOrigins": ["*"],
                        "ExposeHeaders": ["Access-Control-Allow-Origin"],
                    }
                ]
            },
        )

        print(f"  Successful created s3://{self.bucket_name}")
        return response1

    def step_1b_delete_s3_bucket(self) -> dict:
        print("Delete HIL data s3 bucket")
        print(f"  Preview at {self.bucket_console_url}")

        s3dir = S3Path(self.bucket_name)
        s3dir.delete_if_exists()
        # ref: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
        response = self.s3_client.delete_bucket(
            Bucket=self.bucket_name,
        )
        print(f"  Successful deleted s3://{self.bucket_name}")
        return response

    # --- Create IAM Role for HIL
    @property
    def flow_execution_role_name(self) -> str:
        return f"{self.project_name_slug}-flow-role"

    @property
    def flow_execution_role_policy_name(self) -> str:
        return f"{self.project_name_slug}-flow-role-in-line-policy"

    @property
    def flow_execution_role_arn(self) -> str:
        return f"arn:aws:iam::{self.bsm.aws_account_id}:role/{self.flow_execution_role_name}"

    @property
    def flow_execution_role_console_url(self) -> str:
        return f"https://console.aws.amazon.com/iamv2/home?region={self.bsm.aws_region}#/roles/details/{self.flow_execution_role_name}?section=permissions"

    def step_2a_create_flow_execution_role(self) -> dict:
        print("Create IAM role for Human review workflow")
        print(f"  Preview at {self.flow_execution_role_console_url}")
        # ref: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.Client.create_role
        response1 = self.iam_client.create_role(
            RoleName=self.flow_execution_role_name,
            AssumeRolePolicyDocument=json.dumps({
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "sagemaker.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }),
            Tags=self.common_tags,
        )

        # ref: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.Client.put_role_policy
        response2 = self.iam_client.put_role_policy(
            RoleName=self.flow_execution_role_name,
            PolicyName=self.flow_execution_role_policy_name,
            PolicyDocument=json.dumps({
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Action": [
                            "s3:ListBucket",
                            "s3:GetObject",
                            "s3:PutObject",
                            "s3:DeleteObject"
                        ],
                        "Resource": [
                            f"arn:aws:s3:::{self.bucket_name}*"
                        ]
                    }
                ]
            }),
        )

        print(f"  Successful created {self.flow_execution_role_arn}")
        return response1

    def step_2b_delete_flow_execution_role(self) -> dict:
        print("Delete Human review workflow IAM role")
        print(f"  Preview at {self.flow_execution_role_console_url}")
        # ref: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.Client.delete_role_policy
        response = self.iam_client.delete_role_policy(
            RoleName=self.flow_execution_role_name,
            PolicyName=self.flow_execution_role_policy_name,
        )

        # ref: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.Client.delete_role
        response = self.iam_client.delete_role(
            RoleName=self.flow_execution_role_name,
        )

        print(f"  Successful delete {self.flow_execution_role_arn}")
        return response

    # --- Create Task
    @property
    def task_template_name(self) -> str:
        return f"{self.project_name_slug}"

    @property
    def task_template_arn(self) -> str:
        return f"arn:aws:sagemaker:{self.bsm.aws_region}:{self.bsm.aws_account_id}:human-task-ui/{self.task_template_name}"

    @property
    def task_template_console_url(self) -> str:
        return f"https://console.aws.amazon.com/a2i/home?region={self.bsm.aws_region}#/worker-task-templates/{self.task_template_name}"

    def step_3a_create_hil_task_template(self) -> dict:
        print("Create Human in Loop task template")
        print(f"  Preview at {self.task_template_console_url}")
        # ref: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_human_task_ui
        liquid_template = self.path_task_template.read_text(encoding="utf-8")
        response = self.sm_client.create_human_task_ui(
            HumanTaskUiName=self.task_template_name,
            UiTemplate={
                "Content": liquid_template
            },
            Tags=self.common_tags,
        )

        print(f"  Successful created {self.task_template_name}")
        return response

    def step_3b_delete_hil_task_template(self) -> dict:
        print("Delete Human in Loop task template")
        print(f"  Verify at {self.task_template_console_url}")
        # ref: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.delete_human_task_ui
        response = self.sm_client.delete_human_task_ui(
            HumanTaskUiName=self.task_template_name
        )

        print(f"  Successful delete {self.task_template_arn}")
        return response

    # --- Create Human review workflow
    @property
    def flow_definition_name(self) -> str:
        return f"{self.project_name_slug}"

    @property
    def flow_definition_arn(self) -> str:
        return f"arn:aws:sagemaker:{self.bsm.aws_region}:{self.bsm.aws_account_id}:flow-definition/{self.flow_definition_name}"

    @property
    def flow_definition_console_url(self) -> str:
        return f"https://console.aws.amazon.com/a2i/home?region={self.bsm.aws_region}#/human-review-workflows/{self.flow_definition_name}"

    @property
    def s3dir_hil_input(self) -> S3Path:
        return S3Path.from_s3_uri(f"s3://{self.bucket_name}/hil/input").to_dir()

    @property
    def s3dir_hil_output(self) -> S3Path:
        return S3Path.from_s3_uri(f"s3://{self.bucket_name}/hil/output").to_dir()

    def step_4a_create_flow_definition(self) -> dict:
        print("Create Human review workflow definition, it may takes 30 sec ~ 1 minute")
        print(f"  Preview at {self.flow_definition_console_url}")
        # ref: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_flow_definition
        response = self.sm_client.create_flow_definition(
            FlowDefinitionName=self.flow_definition_name,
            HumanLoopConfig={
                "WorkteamArn": self.labeling_team_arn,
                "HumanTaskUiArn": self.task_template_arn,
                "TaskTitle": self.task_template_name,
                "TaskDescription": f"{self.task_template_name} description",
                "TaskCount": 1,  # if it
                "TaskTimeLimitInSeconds": 3600,
            },
            OutputConfig={
                "S3OutputPath": self.s3dir_hil_output.to_file().uri,
            },
            RoleArn=self.flow_execution_role_arn,
            Tags=self.common_tags,
        )

        print(f"  Successful created {self.flow_definition_arn}")
        return response

    def step_4b_delete_flow_definition(self) -> dict:
        print("Delete Human review workflow definition, it may takes 30 sec ~ 1 minute")
        print(f"  Preview at {self.flow_definition_console_url}")
        # ref: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.delete_flow_definition
        response = self.sm_client.delete_flow_definition(
            FlowDefinitionName=self.flow_definition_name
        )

        print(f"  Successful delete {self.flow_definition_arn}")
        return response

    # --- Start Human in Loop
    @property
    def labeling_workforce_console_url(self) -> str:
        return (
            f"https://{self.bsm.aws_region}.console.aws.amazon.com/sagemaker/"
            f"groundtruth?region={self.bsm.aws_region}#/labeling-workforces"
        )

    def get_hil_console_url(self, hil_id: str) -> str:
        return (
            f"https://{self.bsm.aws_region}.console.aws.amazon.com/a2i/home?"
            f"region={self.bsm.aws_region}#/human-review-workflows/"
            f"{self.flow_definition_name}/human-loops/{hil_id}"
        )

    def start_human_loop(self, input_data: dict):
        print("Start human loop ...")
        print(f"  You can enter the labeling portal from {self.labeling_workforce_console_url}")
        response = self.a2i_client.start_human_loop(
            HumanLoopName=str(uuid.uuid4()),
            FlowDefinitionArn=self.flow_definition_arn,
            HumanLoopInput={
                "InputContent": json.dumps(input_data),
            }
        )
        hil_arn = response["HumanLoopArn"]
        hil_id = hil_arn.split("/")[-1]
        hil_console_url = self.get_hil_console_url(hil_id)
        print(f"  You can preview HIL status at {hil_console_url}")


lab = Lab(
    project_name="a2i-poc",
    labeling_team_arn="arn:aws:sagemaker:us-east-2:669508176277:workteam/private-crowd/my-labeling-team",
    bsm=BotoSesManager(profile_name="aws_data_lab_sanhe_us_east_2"),
)

# lab.step_1a_create_s3_bucket()
# lab.step_1b_delete_s3_bucket()
# lab.step_2a_create_flow_execution_role()
# lab.step_2b_delete_flow_execution_role()
# lab.step_3a_create_hil_task_template()
# lab.step_3b_delete_hil_task_template()
# lab.step_4a_create_flow_definition()
# lab.step_4b_delete_flow_definition()


try Augmented AI samples from [amazon-a2i-sample-task-uis](https://github.com/aws-samples/amazon-a2i-sample-task-uis)

In [19]:
def bounding_box_use_case():
    lab.path_task_template = dir_here / "usecases" / "images"/ "bounding-box" / "task.liquid"
    path_artifact = dir_here / "usecases" / "images"/ "bounding-box" / "cat-and-dog.jpeg"
    s3path_artifact = lab.s3dir_hil_input / "cat-and-dog.jpeg"
    s3path_artifact.upload_file(path_artifact.abspath, overwrite=True)
    print(f"Preview artifacts at {s3path_artifact.console_url}")
    input_data = {
        "taskObject": s3path_artifact.uri
    }

    lab.step_3b_delete_hil_task_template()
    lab.step_3a_create_hil_task_template()
    time.sleep(3)
    lab.start_human_loop(input_data)


bounding_box_use_case()

Preview artifacts at https://console.aws.amazon.com/s3/object/669508176277-us-east-2-a2i-poc?prefix=hil/input/cat-and-dog.jpeg
Start human loop ...
You can enter the labeling portal from https://us-east-2.console.aws.amazon.com/sagemaker/groundtruth?region=us-east-2#/labeling-workforces
You can preview HIL status at https://us-east-2.console.aws.amazon.com/a2i/home?region=us-east-2#/human-review-workflows/a2i-poc/human-loops/de6df3aa-a5e5-41d4-869f-c3b93fde2e29


In [32]:
def bounding_box_hierarchicaluse_case():
    lab.path_task_template = dir_here / "usecases" / "images"/ "bounding-box-hierarchical" / "task.liquid"
    path_artifact = dir_here / "usecases" / "images"/ "bounding-box-hierarchical" / "lisa.png"
    s3path_artifact = lab.s3dir_hil_input / "cat-and-dog.jpeg"
    s3path_artifact.upload_file(path_artifact.abspath, overwrite=True)
    print(f"Preview artifacts at {s3path_artifact.console_url}")
    input_data = {
        "taskObject": s3path_artifact.uri
    }

    lab.step_3b_delete_hil_task_template()
    lab.step_3a_create_hil_task_template()
    time.sleep(3)
    lab.start_human_loop(input_data)


bounding_box_hierarchicaluse_case()

Preview artifacts at https://console.aws.amazon.com/s3/object/669508176277-us-east-2-a2i-poc?prefix=hil/input/cat-and-dog.jpeg
Delete Human in Loop task template
  Verify at https://console.aws.amazon.com/a2i/home?region=us-east-2#/worker-task-templates/a2i-poc
  Successful delete arn:aws:sagemaker:us-east-2:669508176277:human-task-ui/a2i-poc
Create Human in Loop task template
  Preview at https://console.aws.amazon.com/a2i/home?region=us-east-2#/worker-task-templates/a2i-poc
  Successful created a2i-poc
Start human loop ...
  You can enter the labeling portal from https://us-east-2.console.aws.amazon.com/sagemaker/groundtruth?region=us-east-2#/labeling-workforces
  You can preview HIL status at https://us-east-2.console.aws.amazon.com/a2i/home?region=us-east-2#/human-review-workflows/a2i-poc/human-loops/0ea0295f-3a99-46d0-b7f5-034a58ea77a4


### key phrase extraction

In [34]:
def key_phrase_extraction_case():
    lab.path_task_template = dir_here / "usecases" / "text"/ "key-phrase-extraction" / "task.liquid"
    input_data = {
        "taskObject": "Excellent bag and fast shipping! Bag arrived right on time and packaged very well. The bag itself is good quality! Was a bit skeptical ordering this bag off amazon but it's 100% authentic and the best price!"
    }

    lab.step_3b_delete_hil_task_template()
    lab.step_3a_create_hil_task_template()
    time.sleep(3)
    lab.start_human_loop(input_data)


key_phrase_extraction_case()

Delete Human in Loop task template
  Verify at https://console.aws.amazon.com/a2i/home?region=us-east-2#/worker-task-templates/a2i-poc
  Successful delete arn:aws:sagemaker:us-east-2:669508176277:human-task-ui/a2i-poc
Create Human in Loop task template
  Preview at https://console.aws.amazon.com/a2i/home?region=us-east-2#/worker-task-templates/a2i-poc
  Successful created a2i-poc
Start human loop ...
  You can enter the labeling portal from https://us-east-2.console.aws.amazon.com/sagemaker/groundtruth?region=us-east-2#/labeling-workforces
  You can preview HIL status at https://us-east-2.console.aws.amazon.com/a2i/home?region=us-east-2#/human-review-workflows/a2i-poc/human-loops/30cf4969-8ec3-4db0-98b1-fe9edd7d9d1a
